In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the uploaded CSV file
file_path = 'All_Occupations.csv'
occupations_df = pd.read_csv(file_path)

# 1) Count how many occupations are on each AA level based on Job Zone
aa_level_counts = occupations_df['AA Level Based on Job Zone'].value_counts()

# Adding data labels to the bar plot for better clarity

# Visualization: Count of occupations on each AA level with data labels
plt.figure(figsize=(8,6))
ax = aa_level_counts.plot(kind='bar', color=['blue', 'green', 'orange'])

# Add labels
for i in ax.containers:
    ax.bar_label(i, label_type='edge')

plt.title('Count of Occupations by AA Level Based on Job Zone')
plt.xlabel('AA Level')
plt.ylabel('Count of Occupations')
plt.xticks(rotation=0)
plt.show()


In [ ]:
# Loading the dataset
file_path = 'all_industry.csv'
industry_df = pd.read_csv(file_path)

# Filtering for high AA levels
high_aa_industries = industry_df[industry_df['AA Level'] == 'High']

# Grouping by NAIC code to count the number of occupations with high AA
top_10_high_aa_naics = high_aa_industries['NAIC'].value_counts().head(10).reset_index()
top_10_high_aa_naics.columns = ['NAIC', 'Count']

# Adjusting the graph to show NAIC codes instead of industry names
plt.figure(figsize=(12, 8))  # Larger figure size
ax = top_10_high_aa_naics.plot(kind='bar', x='NAIC', y='Count', legend=False, color='blue')
plt.title('Top 10 Industries with High AA', fontsize=16)
plt.ylabel('Count of Occupations with High AA', fontsize=12)
plt.xlabel('NAIC Code', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()

# Adding labels to the bars
for i in ax.containers:
    ax.bar_label(i, label_type='edge', fontsize=10)

plt.show()


In [ ]:
# Filtering for medium AA levels instead of high
medium_aa_industries = industry_df[industry_df['AA Level'] == 'Medium']

# Grouping by NAIC code and counting the number of occurrences of medium AA
top_10_medium_aa_naics = medium_aa_industries.groupby('NAIC').size().nlargest(10).reset_index(name='Count')

# Merging NAIC codes with industry names
top_10_medium_aa_naics = top_10_medium_aa_naics.merge(industry_df[['NAIC', 'Industry Name']].drop_duplicates(), on='NAIC', how='left')

# Visualization for the top 10 NAIC codes with medium AA
plt.figure(figsize=(12,8))  # Larger figure size
ax = top_10_medium_aa_naics.plot(kind='bar', x='NAIC', y='Count', legend=False, color='orange')
plt.title('Top 10 Industries with Medium AA', fontsize=16)
plt.ylabel('Count of Occupations with Medium AA', fontsize=12)
plt.xlabel('NAIC Code', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()

# Adding labels to the bars
for i in ax.containers:
    ax.bar_label(i, label_type='edge', fontsize=10)

plt.show()


In [ ]:
# Filtering for medium AA levels instead of high
low_aa_industries = industry_df[industry_df['AA Level'] == 'Low']

# Grouping by NAIC code and counting the number of occurrences of medium AA
top_10_low_aa_naics = low_aa_industries.groupby('NAIC').size().nlargest(10).reset_index(name='Count')

# Merging NAIC codes with industry names
top_10_low_aa_naics = top_10_low_aa_naics.merge(industry_df[['NAIC', 'Industry Name']].drop_duplicates(), on='NAIC', how='left')

# Visualization for the top 10 NAIC codes with medium AA
plt.figure(figsize=(12,8))  # Larger figure size
ax = top_10_low_aa_naics.plot(kind='bar', x='NAIC', y='Count', legend=False, color='green')
plt.title('Top 10 Industries with Low AA', fontsize=16)
plt.ylabel('Count of Occupations with Low AA', fontsize=12)
plt.xlabel('NAIC Code', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()

# Adding labels to the bars
for i in ax.containers:
    ax.bar_label(i, label_type='edge', fontsize=10)

plt.show()

In [ ]:
# Load the dataset
file_path = 'Knowledge.csv'
knowledge_full_df = pd.read_csv(file_path)

# Assign weights to different knowledge categories based on their group and level of abstraction (1-5 scale)
knowledge_weights_grouped = {
    # Arts and Humanities
    'English Language': 2,
    'Fine Arts': 2,
    'Foreign Language': 2,
    'History and Archeology': 2,
    'Philosophy and Theology': 3,

    # Business and Management
    'Administration and Management': 3,
    'Administrative': 2,
    'Customer and Personal Service': 2,
    'Economics and Accounting': 4,
    'Personnel and Human Resources': 2,
    'Sales and Marketing': 2,

    # Communications
    'Communications and Media': 3,
    'Telecommunications': 3,
    'Education and Training': 3,

    # Engineering and Technology
    'Building and Construction': 3,
    'Computers and Electronics': 5,
    'Design': 3,
    'Engineering and Technology': 5,
    'Mechanical': 4,

    # Health Services
    'Medicine and Dentistry': 5,
    'Therapy and Counseling': 4,

    # Law and Public Safety
    'Law and Government': 3,
    'Public Safety and Security': 3,

    # Manufacturing and Production
    'Food Production': 2,
    'Production and Processing': 3,

    # Mathematics and Science
    'Biology': 5,
    'Chemistry': 5,
    'Geography': 5,
    'Mathematics': 5,
    'Physics': 5,
    'Psychology': 4,
    'Sociology and Anthropology': 3,

    # Transportation
    'Transportation': 2,
}

# Update the 'Weight' column with the new weights using the grouped weight dictionary
knowledge_full_df['Weight'] = knowledge_full_df['Knowledge'].map(knowledge_weights_grouped)

# Replace non-numeric "Importance" values with NaN and drop them
knowledge_full_df['Importance'] = pd.to_numeric(knowledge_full_df['Importance'], errors='coerce')
knowledge_full_df = knowledge_full_df.dropna(subset=['Importance'])

# Calculate the AA score as the product of Importance and Weight
knowledge_full_df['AA_Score'] = knowledge_full_df['Importance'] * knowledge_full_df['Weight']

# Calculate percentiles for AA_Score to determine thresholds for Low, Medium, and High categories
low_threshold = knowledge_full_df['AA_Score'].quantile(0.33)
high_threshold = knowledge_full_df['AA_Score'].quantile(0.67)

# Function to categorize AA levels based on the AA_Score using thresholds
def categorize_aa_score_based_on_value(aa_score):
    if aa_score <= low_threshold:
        return 'Low'
    elif aa_score <= high_threshold:
        return 'Medium'
    else:
        return 'High'

# Apply the new categorization function to determine AA Level based on AA_Score
knowledge_full_df['AA_Level'] = knowledge_full_df['AA_Score'].apply(categorize_aa_score_based_on_value)

# Export the updated dataframe to a CSV file for your review
knowledge_full_df.to_csv('knowledge_with_aa_levels_updated.csv', index=False)

# Display a sample of the final dataset
print(knowledge_full_df[['Knowledge', 'Importance', 'Weight', 'AA_Score', 'AA_Level']].head(10))


In [ ]:
# Redefine the file paths
knowledge_with_aa_levels_path = 'knowledge_with_aa_levels_updated.csv'
industry_data_path = 'all_industry.csv'

# Load the updated knowledge dataset and the new industry dataset
knowledge_df = pd.read_csv(knowledge_with_aa_levels_path)
industry_df = pd.read_csv(industry_data_path)

# Extract SOC code from the URL in the knowledge dataset (after the last dash)
knowledge_df['SOC_Code'] = knowledge_df['URL'].str.extract(r'(\d{2}-\d{4}\.\d{2})$')

# Merge the two datasets on SOC code to bring in the industry information
merged_df = pd.merge(knowledge_df, industry_df, left_on='SOC_Code', right_on='Code', how='inner')

In [ ]:
# Filter the merged dataframe for specific industry categories
healthcare_df = merged_df[merged_df['Industry Name'].str.contains('Health', case=False, na=False)]
financial_df = merged_df[merged_df['Industry Name'].str.contains('Finance|Financial|Banking', case=False, na=False)]
manufacture_df = merged_df[merged_df['Industry Name'].str.contains('Manufacturing|Production', case=False, na=False)]

# Plot the AA Score distribution for Healthcare Industry
plt.figure(figsize=(15, 6))
sns.boxplot(data=healthcare_df, x='Knowledge', y='AA_Score', palette='Blues')
plt.xticks(rotation=90)
plt.xlabel('Knowledge Category')
plt.ylabel('AA Score')
plt.title('Distribution of AA Scores by Knowledge Category in Healthcare Industry')
plt.tight_layout()
plt.show()

# Plot the AA Score distribution for Financial Industry
plt.figure(figsize=(15, 6))
sns.boxplot(data=financial_df, x='Knowledge', y='AA_Score', palette='Greens')
plt.xticks(rotation=90)
plt.xlabel('Knowledge Category')
plt.ylabel('AA Score')
plt.title('Distribution of AA Scores by Knowledge Category in Financial Industry')
plt.tight_layout()
plt.show()

# Plot the AA Score distribution for Manufacturing Industry
plt.figure(figsize=(15, 6))
sns.boxplot(data=manufacture_df, x='Knowledge', y='AA_Score', palette='Oranges')
plt.xticks(rotation=90)
plt.xlabel('Knowledge Category')
plt.ylabel('AA Score')
plt.title('Distribution of AA Scores by Knowledge Category in Manufacturing Industry')
plt.tight_layout()
plt.show()

In [ ]:
# Aggregate AA scores by industry to calculate mean, median, and standard deviation for each industry
industry_agg_df = merged_df.groupby('Industry Name')['AA_Score'].agg(['mean', 'median', 'std']).reset_index()

# Rename the columns for better readability
industry_agg_df.columns = ['Industry Name', 'Mean_AA_Score', 'Median_AA_Score', 'Std_AA_Score']

# Display the aggregated data
industry_agg_df.head()


In [ ]:
# Select features for clustering
features = industry_agg_df[['Mean_AA_Score', 'Median_AA_Score', 'Std_AA_Score']]

# Standardize the data
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Apply K-means clustering (choose 3 clusters for simplicity)
kmeans = KMeans(n_clusters=3, random_state=42)
industry_agg_df['Cluster'] = kmeans.fit_predict(features_scaled)

# Plot the clustering result
plt.figure(figsize=(10, 6))
sns.scatterplot(data=industry_agg_df, x='Mean_AA_Score', y='Std_AA_Score', hue='Cluster', palette='viridis', s=100)
plt.xlabel('Mean AA Score')
plt.ylabel('Standard Deviation of AA Score')
plt.title('Clustering of Industries by Ability to Abstract (AA) Scores')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()


In [ ]:
# Group industries by their cluster assignment
clusters = industry_agg_df.groupby('Cluster')['Industry Name'].apply(list)

# Display the industries in each cluster
for cluster_id, industries in clusters.items():
    print(f"Cluster {cluster_id}:")
    for industry in industries:
        print(f" - {industry}")
    print()

In [ ]:
# Filter the industry dataframe for NAIC codes 52, 62, and 31
filtered_industry_df = industry_df[industry_df['NAIC'].isin([52, 62, 31])]

# Merge the filtered industry dataframe with the knowledge dataframe
data = pd.merge(knowledge_df, filtered_industry_df, left_on='SOC_Code', right_on='Code', how='inner')

# Define the knowledge_weights_grouped dictionary
knowledge_weights_grouped = {
    'English Language': 2,
    'Fine Arts': 2,
    'Foreign Language': 2,
    'History and Archeology': 2,
    'Philosophy and Theology': 3,
    'Administration and Management': 3,
    'Administrative': 2,
    'Customer and Personal Service': 2,
    'Economics and Accounting': 4,
    'Personnel and Human Resources': 2,
    'Sales and Marketing': 2,
    'Communications and Media': 3,
    'Telecommunications': 3,
    'Education and Training': 3,
    'Building and Construction': 4,
    'Computers and Electronics': 5,
    'Design': 4,
    'Engineering and Technology': 5,
    'Mechanical': 4,
    'Medicine and Dentistry': 5,
    'Therapy and Counseling': 4,
    'Law and Government': 4,
    'Public Safety and Security': 3,
    'Food Production': 2,
    'Production and Processing': 3,
    'Biology': 5,
    'Chemistry': 5,
    'Geography': 5,
    'Mathematics': 5,
    'Physics': 5,
    'Psychology': 4,
    'Sociology and Anthropology': 3,
    'Transportation': 2,
}

# Map the knowledge weights to the 'Knowledge' column
data['Knowledge_Weight'] = data['Knowledge'].map(knowledge_weights_grouped)

# Calculate the weighted AA score by multiplying Importance by the assigned weight
data['AA_Score'] = data['Importance'] * data['Knowledge_Weight']

# Group by occupation and calculate the mean AA score for each occupation
aa_by_occupation_weighted = data.groupby('Occupation')['AA_Score'].mean().reset_index()

# Rename the columns for clarity
aa_by_occupation_weighted.columns = ['Occupation', 'Mean_AA_Score']

# Define thresholds for Low, Medium, and High based on quantiles (33% and 66%)
low_threshold = aa_by_occupation_weighted['Mean_AA_Score'].quantile(0.25)
high_threshold = aa_by_occupation_weighted['Mean_AA_Score'].quantile(0.50)

# Function to classify AA levels
def classify_aa_level(mean_aa_score):
    if mean_aa_score <= low_threshold:
        return 'Low'
    elif mean_aa_score <= high_threshold:
        return 'Medium'
    else:
        return 'High'

# Apply the function to classify AA levels
aa_by_occupation_weighted['AA_Level'] = aa_by_occupation_weighted['Mean_AA_Score'].apply(classify_aa_level)

# Save the results to a new CSV file
aa_by_occupation_weighted.to_csv('classified_aa_levels_by_occupation.csv', index=False)

# Display the first few rows of the result
print(aa_by_occupation_weighted.head())
